In [1]:
# Install SDP solver and dependencies, slow in first run should be quicker after (a few minutes)
import sys

!conda install --yes --prefix {sys.prefix} pytest

!conda install -c conda-forge --yes cvxpy


!pip install cvxopt


Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/noamavidan/opt/anaconda3

  added / updated specs:
    - cvxpy


The following packages will be UPDATED:

  conda              pkgs/main::conda-4.10.3-py38hecd8cb5_0 --> conda-forge::conda-4.10.3-py38h50d1736_3


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
# Import packages
import cvxpy as cp
import numpy as np
import cvxopt
from cvxpy.expressions.expression import Expression


In [3]:
def Hmin2(A,B):
    """takes rhoA, rhoB numpy arays and returns H_min(A|B)_rho"""
    rhoA = np.asmatrix(A)
    rhoB = np.asmatrix(B)
    if (rhoA.getH() == rhoA).all() & (rhoB.getH() == rhoB).all():
        n = rhoA.shape[0]
        m = rhoB.shape[0]
        IA = np.identity(n)
        IB = np.identity(m)
        A = cp.Parameter((n,n))
        B = cp.Parameter((m,m))
        A.value = rhoA/np.trace(rhoA)
        B.value = rhoB/np.trace(rhoB)

        AB = cp.kron(A,B)/cp.trace(cp.kron(A,B))


        # Calculate exp(-H_min(A|B))
        # Define and solve the CVXPY problem.
        # Create a hermitian matrix variable.
        XB = cp.Variable((m,m),pos=True) 
        # The operator >> denotes matrix inequality.
        constraints = []
        constraints += [cp.kron(IA,XB) >> AB]

        prob = cp.Problem(cp.Minimize(cp.trace(XB)),      
                          constraints)
        prob.solve()
        return(-np.log2(prob.value))
    else:
        print("rhoA and rhoB must be square and of the same dim")

In [4]:
def Hmin(rhoAB,dimA,dimB):
    """takes rhoAB, numpy array and dimA,dimB and returns H_min(A|B)_rho (rhoAB can be non-normal. it is normalized in th function)"""
    if dimA*dimB != rhoAB.shape[0]:
        return("dimA*dimB != shape(rhoAB)")
    if np.any(np.linalg.eigvals(rhoAB) < 0):
        return("rhoAB is not positive semi-definite, min iegenvalue = ",min(np.linalg.eigvals(rhoAB)))
    n = dimA
    m = dimB
    AB = cp.Parameter((n*m,n*m))
    AB.value = rhoAB/np.trace(rhoAB)
    IA = np.identity(n)
    XB = cp.Variable((m,m),pos=True) 
    constraints = []
    constraints += [cp.kron(IA,XB) >> AB]
    prob = cp.Problem(cp.Minimize(cp.trace(XB)),      
                        constraints)
    prob.solve()
    return(-np.log2(prob.value))

In [5]:
def Hmax(rhoAB,dimA,dimB):
    """takes rhoAB, numpy array and dimA,dimB and returns H_max(A|B)_rho (rhoAB can be non-normal. it is normalized in th function)"""
    if dimA*dimB != rhoAB.shape[0]:
        print("dimA*dimB != shape(rhoAB)")
    if np.any(np.linalg.eigvals(rhoAB) < 0):
        print("rhoAB is not positive semi-definite, min iegenvalue = ",min(np.linalg.eigvals(rhoAB)))
    n = dimA
    m = dimB
    AB = cp.Parameter((n*m,n*m))
    AB.value = rhoAB/np.trace(rhoAB)
    IA = np.identity(n)
    X11 = cp.Variable((n*m,n*m))
    X22 = cp.Variable((n*m,n*m)) 
    X12 = cp.Variable((n*m,n*m))
    X21 = cp.Variable((n*m,n*m))
    SigB = cp.Variable((m,m))     
    I11 = np.array([[1,0],[0,0]])
    I12 = np.array([[0,1],[0,0]])
    I21 = np.array([[0,0],[1,0]])
    I22 = np.array([[0,0],[0,1]])
    XAB = cp.kron(I11,X11)+cp.kron(I22,X22)+cp.kron(I12,X12)+cp.kron(I21,X21)

    constraints = []
    constraints += [X11<<AB]
    constraints += [X22<<cp.kron(IA,SigB)]
    constraints += [cp.trace(SigB)<=1]
    constraints += [XAB>>0]
    constraints += [SigB>>0]
    
    prob = cp.Problem(cp.Maximize(0.5*(cp.trace(X12)+cp.trace(X21))),      
                        constraints)
    prob.solve()
    #print(prob.value,XAB.value)
    return(2*np.log2(prob.value))


# Testing random rhoAB's

In [196]:
#defined directly
rhoAB = np.array(([1,1,1,0],
                  [1,12,0,1],
                  [1,0,12,1],
                  [0,1,1,1]))

# defined as tensor product, can be used for LOCC on QC states
#rhoA = np.array(([2,0],
#                [0,1]))


#rhoB = np.array(([1,0],
#                [0,2]))

#rhoAB = np.kron(rhoA,rhoB)

#n = 3
#rhoAB = np.identity(n**2)
#rhoAB = np.zeros((n**2,n**2))
#rhoAB[-1,0] = rhoAB[0,-1]= 1
#rhoAB[0,0] = 10
#rhoAB[-1,-1] = 1
#rhoAB = np.kron(rhoAB,rhoAB)

dimA = int(np.sqrt(rhoAB.shape[0]))
dimB = int(np.sqrt(rhoAB.shape[0]))
print("Hmin=",Hmin(rhoAB,dimA,dimB),"Hmax=",Hmax(rhoAB,dimA,dimB),"dif = ", Hmax(rhoAB,dimA,dimB)-Hmin(rhoAB,dimA,dimB))
#print("rhoAB=\n",rhoAB)

Hmin= 0.10467711061984596 Hmax= 0.5925422642378159 dif =  0.4878651536179699


In [197]:
#looking at the subsystems and at the ratio of the eigenvalues
AB = cp.Parameter((4,4))
AB.value = rhoAB
rhoA = partial_trace(AB,[2,2],axis=1)
rhoA = np.asmatrix(rhoA.value)
rhoB = partial_trace(AB,[2,2],axis=0)
rhoB = np.asmatrix(rhoB.value)
nonzeroiegensAB = [np.linalg.eigvals(rhoAB)[i] for i in range(len(np.linalg.eigvals(rhoAB))) if np.linalg.eigvals(rhoAB)[i] >0.01]
nonzeroiegensA = [np.linalg.eigvals(rhoA)[i] for i in range(len(np.linalg.eigvals(rhoA))) if np.linalg.eigvals(rhoA)[i] >0.01]
nonzeroiegensB = [np.linalg.eigvals(rhoB)[i] for i in range(len(np.linalg.eigvals(rhoB))) if np.linalg.eigvals(rhoB)[i] >0.01]

print("maxA/minA =", max(nonzeroiegensA)/min(nonzeroiegensA))
print("maxB/minB =", max(nonzeroiegensB)/min(nonzeroiegensB))
print("maxAB/minAB =", max(nonzeroiegensAB)/min(nonzeroiegensAB))
print("(maxA/minA)*(maxB/minB)= ", (max(nonzeroiegensB)/min(nonzeroiegensB))*max(nonzeroiegensA)/min(nonzeroiegensA))
rhoB,rhoA



maxA/minA = 1.3636363636363635
maxB/minB = 1.3636363636363635
maxAB/minAB = 19.072568677459696
(maxA/minA)*(maxB/minB)=  1.8595041322314048


(matrix([[13.,  2.],
         [ 2., 13.]]),
 matrix([[13.,  2.],
         [ 2., 13.]]))

## Now I tryed to calculate max_rhoB Hmin(A|B)_rho and min_rhoB Hmax(A|B)_rhoB to see if the difference can be bounded with limitations only on rhoA and rank rho_B (seems like this is not possible, maxHmin can be very big even if B is a small state)

In [ ]:
def Hmin_maxoverB(rhoA,dimB):
    """takes rhoA, rank(rhoB) numpy aray and int and returns max_rhoB H_min(A|B)_rho"""
    if (np.asmatrix(rhoA).getH() == np.asmatrix(rhoA)).all():
        n = rhoA.shape[0]
        m = dimB
        IA = np.identity(n)
        IB = np.identity(m)
        A = cp.Parameter((n,n))
        B = cp.Variable((m,m),PSD = True)
        A.value = rhoA/np.trace(rhoA)
        #B.value = rhoB/np.trace(rhoB)

        #AB = cp.Variable((m*n,m*n))
        AB = cp.kron(A,B)#/cp.trace(cp.kron(A,B))


        # Calculte exp(-H_min(A|B))
        # Define and solve the CVXPY problem.
        # Create a hermitian matrix variable.
        XB = cp.Variable((m,m),pos=True) 
        # The operator >> denotes matrix inequality.


        constraints = []
        constraints += [cp.kron(IA,XB) >> AB] # cp.kron(A,B)*(1/cp.trace(cp.kron(A,B)))] #I*X >= rho_{AB} = A*B
        constraints += [cp.trace(AB)<=1]
        constraints += [cp.trace(B)<=1]
        #constraints += [B<<0.0005*IB]
        #constraints += [B>>0.25*IB]

        prob = cp.Problem(cp.Minimize(cp.trace(XB)),      
                          constraints)
        prob.solve()
        print("B=",B.value)
        print(prob.value)
        return(-np.log2(prob.value))
    else:
        print("rhoA and rhoB must be squere and of the same dim")

In [ ]:
dimB = 3
rhoA = np.identity(4)
rhoB = np.identity(dimB)
Hmin_maxoverB(rhoA,dimB)-Hmin(rhoA,rhoB)


In [111]:
# implament partial trace for min Hmax
def expr_as_np_array(cvx_expr):
    if cvx_expr.is_scalar():
        return np.array(cvx_expr)
    elif len(cvx_expr.shape) == 1:
        return np.array([v for v in cvx_expr])
    else:
        # then cvx_expr is a 2d array
        rows = []
        for i in range(cvx_expr.shape[0]):
            row = [cvx_expr[i,j] for j in range(cvx_expr.shape[1])]
            rows.append(row)
        arr = np.array(rows)
        return arr


def np_array_as_expr(np_arr):
    aslist = np_arr.tolist()
    expr = cp.bmat(aslist)
    return expr


def np_partial_trace(rho, dims, axis=0):
    """
    Takes partial trace over the subsystem defined by 'axis'
    rho: a matrix
    dims: a list containing the dimension of each subsystem
    axis: the index of the subsytem to be traced out
    (We assume that each subsystem is square)
    """
    dims_ = np.array(dims)
    # Reshape the matrix into a tensor with the following shape:
    # [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    # Each subsystem gets one index for its row and another one for its column
    reshaped_rho = np.reshape(rho, np.concatenate((dims_, dims_), axis=None))

    # Move the subsystems to be traced towards the end
    reshaped_rho = np.moveaxis(reshaped_rho, axis, -1)
    reshaped_rho = np.moveaxis(reshaped_rho, len(dims)+axis-1, -1)

    # Trace over the very last row and column indices
    traced_out_rho = np.trace(reshaped_rho, axis1=-2, axis2=-1)

    # traced_out_rho is still in the shape of a tensor
    # Reshape back to a matrix
    dims_untraced = np.delete(dims_, axis)
    rho_dim = np.prod(dims_untraced)
    return traced_out_rho.reshape([rho_dim, rho_dim])


def partial_trace(rho, dims, axis=0):
    if not isinstance(rho, Expression):
        rho = cp.Constant(shape=rho.shape, value=rho)
    rho_np = expr_as_np_array(rho)
    traced_rho = np_partial_trace(rho_np, dims, axis)
    traced_rho = np_array_as_expr(traced_rho)
    return traced_rho



"""
Test out the partial_trace numpy module by creating a matrix
rho_ABC = rho_A \otimes rho_B \otimes rho_C
Each rho_i is normalized, i.e. Tr(rho_i) = 1
"""

# Generate five test cases
rho_A = np.random.rand(4, 4) + 1j*np.random.rand(4, 4)
rho_A /= np.trace(rho_A)
rho_B = np.random.rand(3, 3) + 1j*np.random.rand(3, 3)
rho_B /= np.trace(rho_B)
rho_C = np.random.rand(2, 2) + 1j*np.random.rand(2, 2)
rho_C /= np.trace(rho_C)
rho_AB = np.kron(rho_A, rho_B)
rho_AC = np.kron(rho_A, rho_C)

# Construct a cvxpy Variable with value equal to rho_A \otimes rho_B \otimes rho_C.
temp = np.kron(rho_AB, rho_C)
rho_ABC = cp.Variable(shape=temp.shape, complex=True)
rho_ABC.value = temp

# Try to recover simpler tensors products by taking partial traces of
# more complicated tensors.
rho_AB_test = partial_trace(rho_ABC, [4, 3, 2], axis=2)
rho_AC_test = partial_trace(rho_ABC, [4, 3, 2], axis=1)
rho_A_test = partial_trace(rho_AB_test, [4, 3], axis=1)
rho_B_test = partial_trace(rho_AB_test, [4, 3], axis=0)
rho_C_test = partial_trace(rho_AC_test, [4, 2], axis=0)

# See if the outputs of partial_trace are correct
print("rho_AB test correct? ", np.allclose(rho_AB_test.value, rho_AB))
print("rho_AC test correct? ", np.allclose(rho_AC_test.value, rho_AC))
print("rho_A test correct? ", np.allclose(rho_A_test.value, rho_A))
print("rho_B test correct? ", np.allclose(rho_B_test.value, rho_B))
print("rho_C test correct? ", np.allclose(rho_C_test.value, rho_C))

rho_AB test correct?  True
rho_AC test correct?  True
rho_A test correct?  True
rho_B test correct?  True
rho_C test correct?  True


In [ ]:
# Hmax(A|B) primal

def Hmax_minB(rhoA, dimB):
    n = rhoA.shape[0]
    m = dimB
    A = cp.Parameter((n,n))
    A.value = rhoA
    B = cp.Variable((m,m))
    AB = cp.kron(A,B)#/cp.trace(cp.kron(A,B))
    Y11 = cp.Variable((n*m,n*m),pos=True)
    Y22 = cp.Variable((n*m,n*m),pos=True) 
    Y12 = cp.Variable((n*m,n*m))
    Y21 = cp.Variable((n*m,n*m)) 
    I = np.identity(n)

    I11 = np.matrix([[1,0],[0,0]])
    I12 = np.matrix([[0,1],[0,0]])
    I21 = np.matrix([[0,0],[1,0]])
    I22 = np.matrix([[0,0],[0,1]])
    YAB = cp.kron(I11,Y11)+cp.kron(I22,Y22)+cp.kron(I12,Y12)+cp.kron(I21,Y21)
    II = np.identity(n*m)

    constraints = []
    constraints += [cp.kron(I11,Y11)+cp.kron(I22,Y22) >> 0.5*(cp.kron(I12,II)+cp.kron(I21,II))] 
    constraints += [gamma >= 0] 
    constraints += [cp.trace(AB) <= 1] 
    constraints += [gamma*I >> partial_trace(Y22,[n,m],axis = 1)] #should be gamma*I_B >> partial_trace_{A}(Y22)
    prob = cp.Problem(cp.Minimize(cp.trace(AB*(Y11))+gamma),      
                    constraints)
    prob.solve()
    return(2*np.log2(prob.value))


In [ ]:
dimB = 4
rhoA = np.identity(3)
rhoB = np.identity(dimB)
Hmax(rhoA,rhoB)-Hmax_minB(rhoA,dimB)
